In [14]:
import dill
import pickle
import numpy as np

In [15]:
#Q5 part a 
fx = dill.loads(pickle.load(open('/content/f5.pkl','rb')))
grad = dill.loads(pickle.load(open('/content/grad_f5.pkl','rb')))
hess = dill.loads(pickle.load(open('/content/hess_f5.pkl','rb')))

In [99]:
#Q5 part b
b = np.array([0.76, 0.08, 1.12, 0.68])
Q = np.array([[0.78, -0.02, -0.12, -0.14],[-0.02,0.86,-0.04,0.06],[-0.12,-0.04,0.72,-0.08],[-0.14,0.06,-0.08,0.74]])

def hess():
  return Q
def grad(x):
  return np.matmul(hess(),x) - b
def fx(x):
  return 0.5 * np.sum(x * np.matmul(hess(),x)) - np.sum(b*x)

In [106]:
def l2_norm(x):
    x = np.array(x)
    return np.sqrt(np.sum(x * x))

def grad_descent(X0, alpha=None, eps=None, method='Gradient descent'):
    X=X0
    grad_x=np.array(grad(X))
    k=0
    while(l2_norm(grad_x)>eps):
        k+=1
        alpha = np.sum(grad_x*grad_x)/(np.sum(grad_x * np.matmul(hess(),grad_x)))
        X=X-alpha*grad_x
        grad_x=np.array(grad(X))
        print(f'iter {k} : f(x)={fx(X)} grad(x)={l2_norm(grad(X))}')
    print(f'final f(x) at {k}th iter : {fx(X)}')
    return k

In [ ]:
#X0 = np.array([0, 10])   #Q5 Part a
X0 = np.array([0,0,0,0])  #Q5 Part b
k = grad_descent(X0, eps = 10**(-6))

In [ ]:
#Verify result
x_opt = np.matmul(np.linalg.inv(hess()), b)
print(f'X_true = {x_opt}')
print(f'f(X_true) = {fx(x_opt)}')

# Q6 Inexact line search 

In [145]:
#Q6 Backtracking inexact line search
fx = dill.loads(pickle.load(open('/content/f6.pkl','rb')))
grad = dill.loads(pickle.load(open('/content/grad_f6.pkl','rb')))

In [142]:
def backtracking(X0, alpha=0.5, beta=0.5, eps=10**(-7)):
    X = X0
    u = -1*grad(X)
    t=1
    k=0
    while(l2_norm(t*u)>eps):
      u= -1*grad(X)
      t= 1
      while(fx(X+t*u)>(fx(X)+alpha * t* np.sum(grad(x)*u))):
        t = t*beta
      X = X + t*u
      k+=1
      print(f'iter {k} : f(x)={fx(X)} grad(x)={l2_norm(grad(X))}')
    print(f'final f(x) at {k}th iter : {fx(X)}')
    return [k, X, fx(X)]


In [ ]:
X0 = np.array([10,100,100,10])
print(grad(x))
backtracking(X0, alpha=0.5, beta=0.5, eps=10**(-7))
print(k, X, fx(X))

# Q7 Golden section search and Fibonacci search

In [185]:
def foo(x):
  return np.exp(-x)-np.cos(x)

def GS_Search(a, b, eps):
  xl = a
  xu = b
  k = 0
  l = 0.5* (1+ np.sqrt(5))
  r = l/(1+l)
  tol = 0.5*(xu - xl)
  while(tol>eps):
    d = (xu - xl)*r
    xn = xu - d
    xp = xl + d
    if(foo(xn)<foo(xp)):
      xu = xp
    else:
      xl = xn
    k+=2
    tol = 0.5*(xu - xl)
  return [0.5*(xu+xl), tol, k]
    
def Fibo_Search(a, b, N):
  xl = a
  xu = b
  k = 0
  f = np.zeros((N+1))
  f[0]=1
  f[1]=1
  for j in range(2,N+1):
    f[j]=f[j-1]+f[j-2]

  for  i in range(1,N):
    #print(i,N-i,N-i+1)
    r = f[N-i]/f[N-i+1]
    d = (xu - xl)*r
    xn = xu - d
    xp = xl + d
    if(foo(xn)<foo(xp)):
      xu = xp
    else:
      xl = xn
    k+=2
    tol = 0.5*(xu - xl)
  return [0.5*(xu+xl), tol, k]



In [191]:
x_opt, tol, nsc_20 = Fibo_Search(0,1,20)
print(f'Fibo N= 20: {x_opt, tol, nsc_20}')
x_opt, tol, nsc = GS_Search(0,1,tol)
print(f'GS for tol from N=20: {x_opt, tol, nsc}')

x_opt, tol, nsc_10 = Fibo_Search(0,1,10)
print(f'Fibo N= 10: {x_opt, tol, nsc_10}')
x_opt, tol, nsc = GS_Search(0,1,tol)
print(f'GS for tol from N=10: {x_opt, tol, nsc}')

Fibo N= 20: (0.5885711675497899, 4.567878677141923e-05, 38)
GS for tol from N=20: (0.588543223912219, 3.30534806759486e-05, 40)
Fibo N= 10: (0.5898876404494382, 0.005617977528089846, 18)
GS for tol from N=10: (0.5876574443791536, 0.004065309377891679, 20)
